In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("AAAI 2024/kidsInMindSubtitles2004 (2).csv")
df.shape

(2004, 15)

In [4]:
import openai

In [5]:
openai.api_key = ""

In [6]:
texts = df['subtitles'].tolist()
labels_regression = df['Language'].tolist()

In [7]:
def shorten_subtitle(subtitle, max_tokens):
    if len(subtitle) <= max_tokens:
        return subtitle
    
    words = subtitle.split()

    shortened_subtitle = ""

    for word in words:
        if len(shortened_subtitle) + len(word) + 1 <= max_tokens:
            shortened_subtitle += word + " "
        else:
            break

    shortened_subtitle = shortened_subtitle.strip()

    return shortened_subtitle

In [39]:
def generate_language_score(subtitle):
    # Language model prompt for GPT-3.5
    
    prompt = f"Given the subtitle: '{subtitle}', only print the language rating for the given subtitle in the range of 1 to 10 indicating the proportion and context of profane words"
    
    conversation = [
        {"role": "system", "content": "You are a language rater."},
        {"role": "user", "content": prompt}
    ]
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,  # Set a reasonable value for max_tokens
        temperature=0.7,  # Adjust this parameter to control the response randomness
    )

    # Extract the predicted language score from the model's response
    predicted_score = response.choices[0].text.strip()

    return predicted_score

In [42]:
shortened_subtitle = shorten_subtitle(texts[1803], 2000)
predicted_score = generate_language_score(shortened_subtitle)

In [44]:
import time

In [ ]:
predicted_scores=[]

In [54]:
for text in texts[1943:]:
    shortened_subtitle = shorten_subtitle(text, 2000)
    predicted_score = generate_language_score(shortened_subtitle)
    predicted_scores.append(predicted_score)
    time.sleep(20)

In [59]:
len(predicted_scores[0:201])

201

In [65]:
df_predictions_chatgpt_regression=pd.DataFrame({"Ground_Truth":labels_regression[1803:],"Predictions":predicted_scores[0:201]})

In [66]:
df_predictions_chatgpt_regression.to_excel("predictions_chatgpt_regression.xlsx")

In [67]:
import numpy as np

In [68]:
df_new=pd.read_excel("predictions_chatgpt_regression.xlsx")

In [69]:
df_new.head()

,Unnamed: 0,Ground_Truth,Predictions,Numeric Value
0,0,6,and phrases.\n\nRating: 0,0
1,1,7,used.\n\n7,7
2,2,7,in the subtitle.\n\nRating: 0,0
3,3,9,.\n\n7,7
4,4,1,.\n\nLanguage rating: 7,7


In [75]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
true_labels=labels_regression[1803:]
predictions=df_new['Numeric Value'].tolist()
mse = mean_squared_error(true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_labels, predictions)
r2 = r2_score(true_labels, predictions)
evs = explained_variance_score(true_labels, predictions)


print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("r2: ",r2)
print("evs: ",evs)

Mean Squared Error: 18.99502487562189
Root Mean Squared Error: 4.3583282202722975
Mean Absolute Error: 3.6616915422885574
r2:  -1.446125305997552
evs:  -1.2500828743370054
